# Trading Bot Data Analysis

This notebook performs:
1. Raw data export from database
2. Data cleaning and validation
3. Feature engineering
4. Performance analysis and reporting

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from trading_data.database import TradingDatabase
import warnings
warnings.filterwarnings('ignore')

## 1. Data Export

In [3]:
# Export raw trades from database
db = TradingDatabase()

df = pd.read_sql_query('''
    SELECT 
        id,
        trade_id,
        timestamp,
        action,
        symbol,
        price,
        quantity,
        pnl,
        balance,
        sma_short,
        sma_long
    FROM trades
    ORDER BY timestamp
''', db.conn)

df.to_csv('data/raw_trades.csv', index=False)

print("="*60)
print("Raw Data Overview")
print("="*60)
print(f"Total records: {len(df)}")
print(f"Trading pair: {df['symbol'].unique()}")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nAction distribution:")
print(df['action'].value_counts())
print(f"\nData completeness:")
print(df.isnull().sum())
print(f"\nPrice statistics:")
print(df['price'].describe())

Raw Data Overview
Total records: 47
Trading pair: ['BTCUSDT']
Time range: 2025-10-04T00:35:03.545612 to 2025-10-04T08:09:47.995275

Action distribution:
action
BUY     24
SELL    23
Name: count, dtype: int64

Data completeness:
id            0
trade_id      0
timestamp     0
action        0
symbol        0
price         0
quantity      0
pnl          24
balance       0
sma_short     0
sma_long      0
dtype: int64

Price statistics:
count        47.000000
mean     122488.506809
std         346.598035
min      121992.600000
25%      122305.825000
50%      122448.490000
75%      122587.785000
max      123758.020000
Name: price, dtype: float64


## 2. Data Cleaning

In [4]:
# Load and clean data
df = pd.read_csv('data/raw_trades.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print("\nStarting data cleaning...")

# Fill missing PnL values (BUY actions have no PnL)
df.loc[df['action'] == 'BUY', 'pnl'] = 0

# Detect outliers using IQR method
def detect_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] < lower) | (data[column] > upper)]

# Check price outliers
price_outliers = detect_outliers(df, 'price')
print(f"\nPrice outliers: {len(price_outliers)} records")
if len(price_outliers) > 0:
    print(price_outliers[['timestamp', 'action', 'price']])

# Check PnL outliers (SELL only)
sell_df = df[df['action'] == 'SELL'].copy()
pnl_outliers = detect_outliers(sell_df, 'pnl')
print(f"\nPnL outliers: {len(pnl_outliers)} records")
if len(pnl_outliers) > 0:
    print(pnl_outliers[['timestamp', 'price', 'pnl']])

# Validate trade logic
def validate_trades(data):
    issues = []
    actions = data['action'].tolist()
    
    # Check BUY/SELL pairing
    for i in range(len(actions) - 1):
        if actions[i] == actions[i+1]:
            issues.append(f"Consecutive {actions[i]} at index {i}, {i+1}")
    
    return issues

issues = validate_trades(df)
if issues:
    print("\nValidation issues:")
    for issue in issues:
        print(f"  - {issue}")

# Clean data
df_clean = df[df['price'] > 0].copy()
df_clean.to_csv('data/clean_trades.csv', index=False)
print(f"\nCleaning complete: {len(df_clean)}/{len(df)} records retained")


Starting data cleaning...

Price outliers: 3 records
                    timestamp action      price
0  2025-10-04 00:35:03.545612    BUY  123758.02
1  2025-10-04 00:40:04.885626   SELL  123750.12
22 2025-10-04 03:39:37.953444    BUY  123049.92

PnL outliers: 2 records
                    timestamp      price      pnl
11 2025-10-04 02:32:08.735435  122419.97  0.42737
23 2025-10-04 03:40:08.450037  122671.20 -0.37872

Cleaning complete: 47/47 records retained


## 3. Feature Engineering

In [5]:
# Generate features
df = pd.read_csv('data/clean_trades.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print("\nStarting feature engineering...")

# Time features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Trade pairing features
buy_sell_pairs = []
for i in range(0, len(df) - 1, 2):
    if i+1 < len(df) and df.iloc[i]['action'] == 'BUY' and df.iloc[i+1]['action'] == 'SELL':
        buy_time = df.iloc[i]['timestamp']
        sell_time = df.iloc[i+1]['timestamp']
        hold_duration = (sell_time - buy_time).total_seconds() / 60
        buy_price = df.iloc[i]['price']
        sell_price = df.iloc[i+1]['price']
        
        buy_sell_pairs.append({
            'buy_time': buy_time,
            'sell_time': sell_time,
            'hold_duration_min': hold_duration,
            'buy_price': buy_price,
            'sell_price': sell_price,
            'pnl': df.iloc[i+1]['pnl'],
            'return_pct': ((sell_price - buy_price) / buy_price) * 100
        })

pairs_df = pd.DataFrame(buy_sell_pairs)

# Technical indicator features
df['sma_cross_strength'] = df['sma_short'] - df['sma_long']
df['sma_cross_pct'] = (df['sma_cross_strength'] / df['sma_long']) * 100

# Cumulative features (SELL only)
sell_df = df[df['action'] == 'SELL'].copy()
sell_df['cumulative_pnl'] = sell_df['pnl'].cumsum()
sell_df['running_max'] = sell_df['cumulative_pnl'].cummax()
sell_df['drawdown'] = sell_df['running_max'] - sell_df['cumulative_pnl']

# Rolling statistics
sell_df['pnl_ma5'] = sell_df['pnl'].rolling(window=5, min_periods=1).mean()
sell_df['pnl_std5'] = sell_df['pnl'].rolling(window=5, min_periods=1).std()

# Win/loss streaks
sell_df['is_win'] = (sell_df['pnl'] > 0).astype(int)

# Save engineered features
df.to_csv('data/trades_with_features.csv', index=False)
pairs_df.to_csv('data/buy_sell_pairs.csv', index=False)
sell_df.to_csv('data/sell_trades_features.csv', index=False)

print(f"\nFeature engineering complete")
print(f"Trade pairs: {len(pairs_df)}")
print(f"New features added: {len(df.columns) - 11}")


Starting feature engineering...

Feature engineering complete
Trade pairs: 23
New features added: 5


## 4. Performance Analysis Report

In [6]:
# Generate comprehensive report
pairs_df = pd.read_csv('data/buy_sell_pairs.csv')
sell_df = pd.read_csv('data/sell_trades_features.csv')

print("\n" + "="*60)
print("TRADING PERFORMANCE REPORT")
print("="*60)

# Basic metrics
total_trades = len(pairs_df)
wins = len(pairs_df[pairs_df['pnl'] > 0])
losses = len(pairs_df[pairs_df['pnl'] < 0])
win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0

print(f"\nBasic Metrics:")
print(f"  Total Trades: {total_trades}")
print(f"  Winning Trades: {wins} ({win_rate:.1f}%)")
print(f"  Losing Trades: {losses}")

# PnL analysis
total_pnl = pairs_df['pnl'].sum()
avg_win = pairs_df[pairs_df['pnl'] > 0]['pnl'].mean() if wins > 0 else 0
avg_loss = pairs_df[pairs_df['pnl'] < 0]['pnl'].mean() if losses > 0 else 0
profit_factor = abs(avg_win / avg_loss) if avg_loss != 0 else 0

print(f"\nPnL Analysis:")
print(f"  Total PnL: ${total_pnl:.4f}")
print(f"  Average Win: ${avg_win:.4f}")
print(f"  Average Loss: ${avg_loss:.4f}")
print(f"  Profit Factor: {profit_factor:.2f}")

# Holding period
avg_hold = pairs_df['hold_duration_min'].mean()
max_hold = pairs_df['hold_duration_min'].max()
min_hold = pairs_df['hold_duration_min'].min()

print(f"\nHolding Period:")
print(f"  Average: {avg_hold:.1f} minutes")
print(f"  Maximum: {max_hold:.1f} minutes")
print(f"  Minimum: {min_hold:.1f} minutes")

# Risk metrics
max_dd = sell_df['drawdown'].max()
sharpe_ratio = (sell_df['pnl'].mean() / sell_df['pnl'].std()) * np.sqrt(len(sell_df)) if sell_df['pnl'].std() != 0 else 0

print(f"\nRisk Metrics:")
print(f"  Max Drawdown: ${max_dd:.4f}")
print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")

# Time-based analysis
sell_df['hour'] = pd.to_datetime(sell_df['timestamp']).dt.hour
hourly_stats = sell_df.groupby('hour')['pnl'].agg(['mean', 'count'])

print(f"\nBest Performing Hours (Top 3):")
top_hours = hourly_stats.sort_values('mean', ascending=False).head(3)
for hour, row in top_hours.iterrows():
    print(f"  Hour {hour:02d}: ${row['mean']:.4f} (n={int(row['count'])})")

# Return distribution
return_stats = pairs_df['return_pct'].describe()
print(f"\nReturn Distribution:")
print(f"  Mean: {return_stats['mean']:.2f}%")
print(f"  Std: {return_stats['std']:.2f}%")
print(f"  Min: {return_stats['min']:.2f}%")
print(f"  Max: {return_stats['max']:.2f}%")

print("\n" + "="*60)
print("Analysis complete. Data saved to:")
print("  - data/raw_trades.csv")
print("  - data/clean_trades.csv")
print("  - data/trades_with_features.csv")
print("  - data/buy_sell_pairs.csv")
print("  - data/sell_trades_features.csv")
print("="*60)


TRADING PERFORMANCE REPORT

Basic Metrics:
  Total Trades: 23
  Winning Trades: 9 (39.1%)
  Losing Trades: 14

PnL Analysis:
  Total PnL: $-0.1749
  Average Win: $0.1336
  Average Loss: $-0.0984
  Profit Factor: 1.36

Holding Period:
  Average: 7.6 minutes
  Maximum: 29.6 minutes
  Minimum: 0.5 minutes

Risk Metrics:
  Max Drawdown: $0.7796
  Sharpe Ratio: -0.23

Best Performing Hours (Top 3):
  Hour 02: $0.0848 (n=4)
  Hour 06: $0.0839 (n=3)
  Hour 01: $0.0508 (n=3)

Return Distribution:
  Mean: -0.01%
  Std: 0.13%
  Min: -0.31%
  Max: 0.35%

Analysis complete. Data saved to:
  - data/raw_trades.csv
  - data/clean_trades.csv
  - data/trades_with_features.csv
  - data/buy_sell_pairs.csv
  - data/sell_trades_features.csv


In [7]:
# Trading Data Analysis Pipeline
# Complete analysis from raw data export to performance report

# %% [markdown]
# # Trading Bot Data Analysis
# 
# This notebook performs:
# 1. Raw data export from database
# 2. Data cleaning and validation
# 3. Feature engineering
# 4. Performance analysis and reporting

# %% Import Libraries
import pandas as pd
import numpy as np
from trading_data.database import TradingDatabase
import warnings
warnings.filterwarnings('ignore')

# %% [markdown]
# ## 1. Data Export

# %% Export raw trades from database
db = TradingDatabase()

df = pd.read_sql_query('''
    SELECT 
        id,
        trade_id,
        timestamp,
        action,
        symbol,
        price,
        quantity,
        pnl,
        balance,
        sma_short,
        sma_long
    FROM trades
    ORDER BY timestamp
''', db.conn)

# Save raw data
df.to_csv('data/raw_trades.csv', index=False)

print("="*60)
print("Raw Data Overview")
print("="*60)
print(f"Total records: {len(df)}")
print(f"Trading pair: {df['symbol'].unique()}")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nAction distribution:")
print(df['action'].value_counts())
print(f"\nData completeness:")
print(df.isnull().sum())
print(f"\nPrice statistics:")
print(df['price'].describe())

# %% [markdown]
# ## 2. Data Cleaning

# %% Load and clean data
df = pd.read_csv('data/raw_trades.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print("\nStarting data cleaning...")

# Fill missing PnL values (BUY actions have no PnL)
df.loc[df['action'] == 'BUY', 'pnl'] = 0

# Detect outliers using IQR method
def detect_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] < lower) | (data[column] > upper)]

# Check price outliers
price_outliers = detect_outliers(df, 'price')
print(f"\nPrice outliers: {len(price_outliers)} records")
if len(price_outliers) > 0:
    print(price_outliers[['timestamp', 'action', 'price']])

# Check PnL outliers (SELL only)
sell_df = df[df['action'] == 'SELL'].copy()
pnl_outliers = detect_outliers(sell_df, 'pnl')
print(f"\nPnL outliers: {len(pnl_outliers)} records")
if len(pnl_outliers) > 0:
    print(pnl_outliers[['timestamp', 'price', 'pnl']])

# Validate trade logic
def validate_trades(data):
    issues = []
    actions = data['action'].tolist()
    
    # Check BUY/SELL pairing
    for i in range(len(actions) - 1):
        if actions[i] == actions[i+1]:
            issues.append(f"Consecutive {actions[i]} at index {i}, {i+1}")
    
    return issues

issues = validate_trades(df)
if issues:
    print("\nValidation issues:")
    for issue in issues:
        print(f"  - {issue}")

# Clean data
df_clean = df[df['price'] > 0].copy()
df_clean.to_csv('data/clean_trades.csv', index=False)
print(f"\nCleaning complete: {len(df_clean)}/{len(df)} records retained")

# %% [markdown]
# ## 3. Feature Engineering

# %% Generate features
df = pd.read_csv('data/clean_trades.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print("\nStarting feature engineering...")

# Time features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Trade pairing features
buy_sell_pairs = []
for i in range(0, len(df) - 1, 2):
    if i+1 < len(df) and df.iloc[i]['action'] == 'BUY' and df.iloc[i+1]['action'] == 'SELL':
        buy_time = df.iloc[i]['timestamp']
        sell_time = df.iloc[i+1]['timestamp']
        hold_duration = (sell_time - buy_time).total_seconds() / 60
        buy_price = df.iloc[i]['price']
        sell_price = df.iloc[i+1]['price']
        
        buy_sell_pairs.append({
            'buy_time': buy_time,
            'sell_time': sell_time,
            'hold_duration_min': hold_duration,
            'buy_price': buy_price,
            'sell_price': sell_price,
            'pnl': df.iloc[i+1]['pnl'],
            'return_pct': ((sell_price - buy_price) / buy_price) * 100
        })

pairs_df = pd.DataFrame(buy_sell_pairs)

# Technical indicator features
df['sma_cross_strength'] = df['sma_short'] - df['sma_long']
df['sma_cross_pct'] = (df['sma_cross_strength'] / df['sma_long']) * 100

# Cumulative features (SELL only)
sell_df = df[df['action'] == 'SELL'].copy()
sell_df['cumulative_pnl'] = sell_df['pnl'].cumsum()
sell_df['running_max'] = sell_df['cumulative_pnl'].cummax()
sell_df['drawdown'] = sell_df['running_max'] - sell_df['cumulative_pnl']

# Rolling statistics
sell_df['pnl_ma5'] = sell_df['pnl'].rolling(window=5, min_periods=1).mean()
sell_df['pnl_std5'] = sell_df['pnl'].rolling(window=5, min_periods=1).std()

# Win/loss streaks
sell_df['is_win'] = (sell_df['pnl'] > 0).astype(int)

# Save engineered features
df.to_csv('data/trades_with_features.csv', index=False)
pairs_df.to_csv('data/buy_sell_pairs.csv', index=False)
sell_df.to_csv('data/sell_trades_features.csv', index=False)

print(f"\nFeature engineering complete")
print(f"Trade pairs: {len(pairs_df)}")
print(f"New features added: {len(df.columns) - 11}")

# %% [markdown]
# ## 4. Performance Analysis Report

# %% Generate comprehensive report
pairs_df = pd.read_csv('data/buy_sell_pairs.csv')
sell_df = pd.read_csv('data/sell_trades_features.csv')

print("\n" + "="*60)
print("TRADING PERFORMANCE REPORT")
print("="*60)

# Basic metrics
total_trades = len(pairs_df)
wins = len(pairs_df[pairs_df['pnl'] > 0])
losses = len(pairs_df[pairs_df['pnl'] < 0])
win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0

print(f"\nBasic Metrics:")
print(f"  Total Trades: {total_trades}")
print(f"  Winning Trades: {wins} ({win_rate:.1f}%)")
print(f"  Losing Trades: {losses}")

# PnL analysis
total_pnl = pairs_df['pnl'].sum()
avg_win = pairs_df[pairs_df['pnl'] > 0]['pnl'].mean() if wins > 0 else 0
avg_loss = pairs_df[pairs_df['pnl'] < 0]['pnl'].mean() if losses > 0 else 0
profit_factor = abs(avg_win / avg_loss) if avg_loss != 0 else 0

print(f"\nPnL Analysis:")
print(f"  Total PnL: ${total_pnl:.4f}")
print(f"  Average Win: ${avg_win:.4f}")
print(f"  Average Loss: ${avg_loss:.4f}")
print(f"  Profit Factor: {profit_factor:.2f}")

# Holding period
avg_hold = pairs_df['hold_duration_min'].mean()
max_hold = pairs_df['hold_duration_min'].max()
min_hold = pairs_df['hold_duration_min'].min()

print(f"\nHolding Period:")
print(f"  Average: {avg_hold:.1f} minutes")
print(f"  Maximum: {max_hold:.1f} minutes")
print(f"  Minimum: {min_hold:.1f} minutes")

# Risk metrics
max_dd = sell_df['drawdown'].max()
sharpe_ratio = (sell_df['pnl'].mean() / sell_df['pnl'].std()) * np.sqrt(len(sell_df)) if sell_df['pnl'].std() != 0 else 0

print(f"\nRisk Metrics:")
print(f"  Max Drawdown: ${max_dd:.4f}")
print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")

# Time-based analysis
sell_df['hour'] = pd.to_datetime(sell_df['timestamp']).dt.hour
hourly_stats = sell_df.groupby('hour')['pnl'].agg(['mean', 'count'])

print(f"\nBest Performing Hours (Top 3):")
top_hours = hourly_stats.sort_values('mean', ascending=False).head(3)
for hour, row in top_hours.iterrows():
    print(f"  Hour {hour:02d}: ${row['mean']:.4f} (n={int(row['count'])})")

# Return distribution
return_stats = pairs_df['return_pct'].describe()
print(f"\nReturn Distribution:")
print(f"  Mean: {return_stats['mean']:.2f}%")
print(f"  Std: {return_stats['std']:.2f}%")
print(f"  Min: {return_stats['min']:.2f}%")
print(f"  Max: {return_stats['max']:.2f}%")

print("\n" + "="*60)
print("Analysis complete. Data saved to:")
print("  - data/raw_trades.csv")
print("  - data/clean_trades.csv")
print("  - data/trades_with_features.csv")
print("  - data/buy_sell_pairs.csv")
print("  - data/sell_trades_features.csv")
print("="*60)

Raw Data Overview
Total records: 47
Trading pair: ['BTCUSDT']
Time range: 2025-10-04T00:35:03.545612 to 2025-10-04T08:09:47.995275

Action distribution:
action
BUY     24
SELL    23
Name: count, dtype: int64

Data completeness:
id            0
trade_id      0
timestamp     0
action        0
symbol        0
price         0
quantity      0
pnl          24
balance       0
sma_short     0
sma_long      0
dtype: int64

Price statistics:
count        47.000000
mean     122488.506809
std         346.598035
min      121992.600000
25%      122305.825000
50%      122448.490000
75%      122587.785000
max      123758.020000
Name: price, dtype: float64

Starting data cleaning...

Price outliers: 3 records
                    timestamp action      price
0  2025-10-04 00:35:03.545612    BUY  123758.02
1  2025-10-04 00:40:04.885626   SELL  123750.12
22 2025-10-04 03:39:37.953444    BUY  123049.92

PnL outliers: 2 records
                    timestamp      price      pnl
11 2025-10-04 02:32:08.735435  